# 05_compute_ISC

In [1]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack
import pickle

In [2]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
band_names = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# set path
pre_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
post_root = pjoin(pre_root, 'derivatives', 'preproc_meg-mne_mri-fmriprep')

results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [3]:
def extract_megdata(bids_root, sub, run):

    sub_path = BIDSPath(subject=sub, run=run, task='movie', session='movie', root=bids_root)
    raw = mne_bids.read_raw_bids(sub_path)
    
    return raw

In [4]:
def extract_events(bids_root, sub, run):
    
    raw = extract_megdata(bids_root, sub, run)
    events = mne.find_events(raw, stim_channel='UPPT001', min_duration=2/raw.info['sfreq'])
    
    return events

In [5]:
def band_split(raw, band_name):
    """

    """
    band_freq = {'delta': [1, 4],
                 'theta': [4, 8],
                 'alpha': [8, 13],
                 'beta': [13, 30],
                 'gamma': [30, 100]}
    
    raw_band = raw.copy().load_data().filter(l_freq=band_freq[band_name][0], h_freq=band_freq[band_name][1])

    return raw_band

In [6]:
def compute_isc(data1, data2):

    """calculate inter-subject correlation along the determined axis.

    Parameters
    ----------

        data1, data2: array,
            shape = [n_samples, n_features].

    Returns
    -------
        isc: point-to-point functional connectivity list of
            data1 and data2, shape = [n_samples].

    """

    data1 = np.nan_to_num(data1)
    data2 = np.nan_to_num(data2)

    z_data1 = np.nan_to_num(stats.zscore(data1, axis=-1))
    z_data2 = np.nan_to_num(stats.zscore(data2, axis=-1))
    corr = np.sum(z_data1*z_data2, axis=-1)/(np.size(data1, -1))

    return corr

In [7]:
def compute_band_isc(data_sub1, data_sub2, events_sub1, events_sub2, band_name):
    """
    
    """
    
    # get valid channels
    def get_picks(data):
        ch_name_picks = mne.pick_channels_regexp(data.ch_names, regexp='M[LRZ]...-4503')
        type_picks = mne.pick_types(data.info, meg=True)
        picks= np.intersect1d(ch_name_picks, type_picks)
        return picks
        
    picks_sub1 = get_picks(data_sub1)
    picks_sub2 = get_picks(data_sub2)

    # match channels
    picks = np.intersect1d(picks_sub1, picks_sub2)
    
    # mark bad channels
    bad_idx = []
    if len(picks) != len(picks_sub1):
        bad_picks = np.union1d(np.setdiff1d(picks_sub1, picks_sub2), np.setdiff1d(picks_sub2, picks_sub1))
        for chn in bad_picks:
            bad_idx.append(np.where(picks == chn)[0][0])
    
    # band_split
    band_sub1 = band_split(data_sub1, band_name=band_name)
    band_sub2 = band_split(data_sub2, band_name=band_name)
    band_sub1_data = band_sub1.get_data(picks=picks)
    band_sub2_data = band_sub2.get_data(picks=picks)
    
    envlope_sub1 = np.abs(signal.hilbert(band_sub1_data))
    envlope_sub2 = np.abs(signal.hilbert(band_sub2_data))
    
    if len(bad_idx) != 0:
        for idx in bad_idx:
            envlope_sub1[idx,:] = 0
            envlope_sub2[idx,:] = 0
    
    # downsampling
    sample_sub1 = events_sub1[1:-1:25,0]
    sample_sub2 = events_sub2[1:-1:25,0]
    band_dsamp_sub1 = envlope_sub1.take(sample_sub1, axis=1)
    band_dsamp_sub2 = envlope_sub2.take(sample_sub2, axis=1)
    
    isc = compute_isc(band_dsamp_sub1, band_dsamp_sub2)
    del data_sub1, data_sub2
    
    return isc

In [8]:
def compute_bandISC(bids_root, sub_list, run_list, band_names, save_path=None):
    
    bandISC = {band_name : np.zeros((len(run_list), len(sub_list), len(sub_list), 272))
               for band_name in band_names}
    
    for band_name in band_names:

        for run in run_list:
            if int(run) >= 7:
                sub_ls = sub_list[1:]
            else:
                sub_ls = sub_list

            # pair-wise ISC
            for i, sub1 in enumerate(sub_ls):
                megdata_sub1 = extract_megdata(bids_root, sub1, run)
                events_sub1 = extract_events(bids_root, sub1, run)
            
                for sub2 in sub_ls[i+1:]:
                    megdata_sub2 = extract_megdata(bids_root, sub2, run)
                    events_sub2 = extract_events(bids_root, sub2, run)
                    
                    isc = compute_band_isc(megdata_sub1, megdata_sub2, events_sub1, events_sub2, band_name)

                    bandISC[band_name][int(run)-1, int(sub1)-1, int(sub2)-1] = np.asarray(isc)
                    
                    # save 
                    if save_path is not None:
                        with open(save_path, 'wb') as fp:
                            pickle.dump(bandISC, fp)      

    return bandISC

In [ ]:
compute_bandISC(pre_root, sub_list, run_list, band_names, pjoin(results_pth, 'bandISC_pre.pickle'))
compute_bandISC(post_root, sub_list, run_list, band_names, pjoin(results_pth, 'bandISC_post.pickle'))